In [10]:
import json
import pandas as pd
LINKS_FILE = "../data/pdf_links.json"
os.makedirs(os.path.dirname(LINKS_FILE), exist_ok=True)
with open(LINKS_FILE , 'rt') as f_out:
    documents = json.load(f_out)
df = pd.DataFrame(documents)
df = df.to_dict(orient='records')

In [2]:
import requests
import fitz  
from io import BytesIO
from tqdm.auto import tqdm

for doc in df:
    try:
        
        response = requests.get(doc['url'])
        response.raise_for_status()

        
        pdf_file = BytesIO(response.content)

        # Open the PDF and extract text
        pdf = fitz.open(stream=pdf_file, filetype="pdf")
        text = ""
        for page in tqdm(pdf):
            text += page.get_text()
        pdf.close()

        # Add the extracted text to the dictionary
        doc["info"] = text.strip()

    except Exception as e:
        doc["info"] = f"Error extracting text: {e}"


In [11]:
documents[79]

{'url': 'https://nitw.ac.in/api/static/files/AI_OUTR_B_Brouchre-1_2025-5-16-16-32-17.pdf',
 'date': '2025-05-16',
 'title': 'ONLINE FACULTY DEVELOPMENT PROGRAMME (FDP)ON : AI/ML Techniques for Medical Image and Signal Analysis: A Research Oriented Approach | (18th June – 28th June 2025) Organized by Electronics & ICT Academy, NIT Warangal.'}

In [4]:
import re

def clean_text(raw_text):
    text = re.sub(r'\n\s*\n+', '\n\n', raw_text)
    text = re.sub(r'(?<!\n)\n(?!\n)', ' ', text)
    text = re.sub(r' +', ' ', text)

    return text


In [5]:
for doc in df:
    doc['info'] = clean_text(doc['info'])
    

In [6]:
for pos ,doc in enumerate(df):
    if pos in [27 , 28 ,29, 30]:
        print(doc, '\n')
       

{'url': 'https://nitw.ac.in/api/static/files/dvt2024_09-09_2024-9-9-18-14-3.pdf', 'date': '2024-09-03', 'title': 'Recruitment to the post of Registrar at NIT Warangal. Last date for receipt of applications: 13.10.2024 | ', 'info': 'Advt. No. 04/2024 dated 09.09.2024 \n\nPage 1 of 5 NATIONAL INSTITUTE OF TECHNOLOGY WARANGAL (Ministry of Education, Govt of India) WARANGAL – 506 004 (Telangana State) \n\nRecruitment Advt. No. 04/2024 dated 09.09.2024 RECRUITMENT TO THE POST OF REGISTRAR AT NIT WARANGAL “NIT WARANGAL STRIVES TO HAVE A WORKFORCE WHICH REFLECTS GENDER BALANCE AND WOMEN CANDIDATES ARE ENCOURAGED TO APPLY” Online applications are invited from eligible Indian Citizens to fill up the post of REGISTRAR on Deputation (IncludingShort Term Contract) basis. Eligible candidates can apply through the online recruitment portal available at: https://nitw.ac.in/Careers/. Post Code NITW/4/24/Registrar Emoluments Minimum pay of Rs. 1,44,200/- in Level 14 of the 7thCPC + DA + allowances as a

In [7]:
with open("database.json", "w", encoding="utf-8") as f_in:
    json.dump(df, f_in, indent=4, ensure_ascii=False)

In [26]:

response = requests.get('https://nitw.ac.in/api/static/files/Allotment_of_MNC-4_courses_to_B.Tech._II_Year_Students-new_2024-7-30-14-46-27.pdf')
response.raise_for_status()

# Use BytesIO to avoid saving to disk
pdf_file = BytesIO(response.content)

# Open the PDF and extract text
pdf = fitz.open(stream=pdf_file, filetype="pdf")
text = ""
for page in pdf:
    text += page.get_text()
pdf.close()

# Add the extracted text to the dictionary
out = text.strip()
out = clean_text(out)
